In [15]:
%run Lexical_Analyzer.ipynb

In [16]:
%run parser_helpers.ipynb

In [17]:
%run CompilerTheory_Ch5(Tree_Classes).ipynb

In [18]:
%run Translator.ipynb

In [24]:
def pre_process(T):
    global ST
    """traverses the abstract syntax tree T and pre-processes it to make it easier to generate intermediate code"""
    
    def pre_traverse(T,p):
        """traverses the tree and processes it"""
        
        #-----------------------PRE-ORDER ACTIONS---------------------------------------------------------------

        if p.element() == 'IF':    
            if T.num_children(p) == 2:                          # there was no corresponding else statement
                T1 = MutableLinkedTree()
                root = T1.add_root('ELSE')                      # create an ELSE subtree
                T._attacho(p,T1)                                # attach as a third child
                
                
        for pos in T.children(p):
            #-------------------IN-ORDER ACTIONS---------------------------------------------------------------
            pre_traverse(T,pos)
            
        # ----------------------POST-ORDER ACTIONS---------------------------------------------------------------
        """Want to have op as parent of ID so that the ID node is processed before the opaerator node in 
        the post-order actions in int_code_gen so that the ID is in the stack. Pre and post fix are taken 
        care of in the order of the children."""
        if p.element() in ['++', '--']:                        # check for unary operators
            if T.is_leaf(p):                                   # check if the operator is a leaf
                temp = p.element()                             # store p's element for now
                T._replace(p,T.parent(p).element())            # replace p's element with its parent's element - the ID
                T._replace(T.parent(p), temp)                  # replace the parent's element with p's element
        
        if p.element() in ST:                                  # p is an ID
            # p has a child, so is an array, and we haven't updated the array size yet
            if not ST[p.element()].array_size and T.num_children(p) > 0:  
                ST[p.element()].array_size = T.children(p)[0].element()
            
        
    if v: print("-"*100)
    pre_traverse(T,T.root())       # call the recursive code above
    if v: print("-"*100)
    
    return T                       # return the processed tree

In [1]:
def int_code_gen(T,v,vv):
    """traverses the pre-proccessed abstract syntax tree T and produces intermediate code for expressions"""
    global ST
    
    relops = {'<':'>=', '>':'<=', '<=':'>', '>=':'<', '==':'!=', '!=':'=='}   # dictionary for the opposite relop
   
    #-----------------------------------------------------------------------------------------------
    def label(s):
        """returns a string with a number attached for uniqueness"""
        nonlocal count
        
        count += 1
        
        if s == "temp":                                            # add temporary variables to the symbol table
            ST[s + str(count)] = Token(s + str(count), "ID")
        if s == "DO-WHILE":
            return "DO_WHILE" + str(count)
        return s + str(count)
    
    #-----------------------------------------------------------------------------------------------
    def traverse(T,p):
        """traverses the tree producing intermediate code"""
        
        #-------------------------------PRE-ORDER VISIT ACTION------------------------------------------
        if p.element() in ['FOR', 'WHILE', 'DO-WHILE']:            # check if we have a statement node
            label_name = label(p.element())                        # create unique name for label
            out_name = label("out")                                # create a unique output name
            int_code.append(["label",label_name,None,None])        # append quadruple to output
            A.append([p.element(),label_name,out_name])                 # create and append activation record
            if v: print(label_name,":")                            #-----------------------------print output
        
        elif p.element() == 'IF':
            else_label_name = label('ELSE')                        # create unique name for ELSE label 
            out_name = label("out")                                # create a unique output name
            A.append(["ELSE",else_label_name,out_name])            # create and append activation record
            
            label_name = label(p.element())                        # create unique name for IF label
            int_code.append(["label",label_name,None,None])        # append quadruple to output
            A.append(["IF",label_name,out_name])                   # create and append activation record - same out label as ELSE
            if v: print(label_name,":") 
            
        elif p.element() == 'ELSE':
            unconditional = A.pop()                                # get the out lable for IF
            int_code.append(['JUMP',unconditional[2],None,None])   # append the unconditional jump 
            if v: print('goto', unconditional[2])                  #---------------------------print the output
              
            label_name = A[-1][1]                                  # get the else label name but don't take it off the stack
            int_code.append(["label",label_name,None,None])        # append quadruple to output
            if v: print(label_name,":")
            
        #-------------------------------PRE-ORDER VISIT ACTION------------------------------------------
        
        
        
        for pos in T.children(p):       # Recursive call to children of p
            
            #--------------------------------IN-ORDER VISIT ACTION------------------------------------------
            
            traverse(T,pos)
        
        
        
        #------------------------------POST-ORDER VISIT ACTION------------------------------------------
        if p.element() in ['+','-','*','/', '%']: # check for operators 
            right = S.pop()                                                 # pop (most recent) operand
            left = S.pop()                                                  # pop (older) operand
            temp_name = label("temp")                                       # generate temporary label
            
            int_code.append([temp_name,str(left),p.element(),str(right)])   # save Quadruple to output
            if v: print(temp_name,"=",str(left),p.element(),str(right))     #---------------------------print output
            
            S.append(temp_name)                                             # push temporary label to stack
            
        elif p.element() in ['<', '>', '<=', '>=', '==', '!=']:             # check for relative operators 
            right = S.pop()                                                 # pop (most recent) operand
            left = S.pop()                                                  # pop (older) operand
            
            if T.parent(p).element() == 'FOR':                              
                label_name = label('unconditional_jmp')                     # create label name for an unconditional jump 
                int_code.append(["label",label_name,None,None])             # append quadruple to output
                A.append(["expr",None,label_name])                          # create and append activation record
                if v: print(label_name,":")                                 #-----------------------------print output
            
                int_code.append([str(left),relops[p.element()],str(right), A[-2][2]])        # save Quadruple to output
                if v: print('if', str(left),relops[p.element()],str(right), 'goto', A[-2][2])#---------------------------print output
            
            elif T.parent(p).element() == 'IF':
                int_code.append([str(left),relops[p.element()],str(right), A[-2][1]])        # save Quadruple to output
                if v: print('if', str(left),relops[p.element()],str(right), 'goto', A[-2][1])#---------------------------print output
            
            elif T.parent(p).element() == 'DO-WHILE':
                int_code.append([str(left),p.element(),str(right), A[-1][1]])        # save Quadruple to output
                if v: print('if', str(left),p.element(),str(right), 'goto', A[-1][1])#---------------------------print output
                    
            else:                                                           # jump over code if condition is false
                int_code.append([str(left),relops[p.element()],str(right), A[-1][2]])        # save Quadruple to output
                if v: print('if', str(left),relops[p.element()],str(right), 'goto', A[-1][2])#---------------------------print output
                    
        elif p.element() == '=':                                            # check for an equality
            right = S.pop()                                                 # pop the right hand side of the expression
            var = S.pop()                                                   # pop the variable being assigned
            
            int_code.append([var,str(right), None, None])                   # save Quadruple to output
            if v: print(var,"=",str(right))                                 #---------------------------print output
        
        elif p.element() in ['+=', '-=', '*=', '/=']:                       # check for incrementors
            right = S.pop()                                                 # pop the right hand side of the expression
            var = S.pop()                                                   # pop the variable being assigned
            
            int_code.append([var, var, p.element()[0], str(right)])         # save Quadruple to output
            if v: print(var, '=', var,p.element()[0],str(right))            #---------------------------print output
                
        elif p.element() in ['++', '--']:                                   # check for unary operators 
            # implement as x = x + 1
            var = S.pop()                                                   # pop the variable 

            int_code.append([var, var, p.element()[0], '1'])                # save Quadruple to output
            if v: print(var, '=', var,p.element()[0],'1')                   #---------------------------print output
            
        elif p.element() == 'IF':   
            pass                                                            # don't do anything for if nodes here
        
        elif p.element() == 'ELSE':  
            record = A.pop()                                                # pop current activation record
            int_code.append(["label",record[2],None,None])                  # append quadruple to output
            if v: print(record[2],":") 
                
        elif p.element() == 'FOR':
            unconditional = A.pop()                                         # get the lable for the conparison
            int_code.append(['JUMP',unconditional[2],None,None])            # append the unconditional jump 
            if v: print('goto', unconditional[2])                           #---------------------------print the output
            
            record = A.pop()                                                # pop current activation record
            int_code.append(["label",record[2],None,None])                  # append quadruple to output
            if v: print(record[2],":")                                      #---------------------------print output
        
        elif p.element() == 'WHILE':
            record = A.pop()                                                # pop current activation record
            int_code.append(["JUMP",record[1],None,None])                   # append the unconditional jump to the start of the while loop 
            if v: print('goto', record[1])
            int_code.append(["label",record[2],None,None])                  # append quadruple to output
            if v: print(record[2],":")                                      #---------------------------print output
        
        elif p.element() == 'DO-WHILE':
            pass                                                            # don't do anything for do-while loops here
        
        else:
            if T.num_children(p) > 0:                                       # if anything else has children, it is an array expr
                arr_expr = S.pop()                                          # get the expression from the operands stack
                var = p.element() + '[' + str(arr_expr) + ']'
                S.append(var)                                               # push the variable and the array expr onto stack
            else: S.append(p.element())                                     # push non-operator and non-list nodes on the stack
        #------------------------------POST-ORDER VISIT ACTION----------------------------------------------
    
    
    
    
    int_code = []
    count = 0
    S = []                       # stack for operands/results
    A = []                       # stack for activiation records
    
    if v: print("-"*100)
    traverse(T,T.root())         # call the recursive code above
    if v: print("-"*100)
    
    return int_code

In [21]:
def intermediate_code_generator(source,v=False,vv=False):
    """wrapper/driver for intermediate code generator"""
    
    T0 = parser(source)             # parse and translate the source to an AST
    
    T = pre_process(T0)             # pre-process the tree
        
    if vv: printTree(T,1,2000,18)   # print the AST (if very verbose)
    
    int_code = int_code_gen(T,v,vv)
    
    if v: print("\n\nQuadruples")
    return int_code